# 🍎 파이썬 머신러닝 완벽 가이드 혼공

### 2019.07.10 ~ 교재 8장

### 08. 텍스트 분석

## 01. 텍스트 분석 이해 
텍스트를 피처 벡터화 해주어야 하는데 1) BOW 2) Word2Vec 방법을 사용할 수 있다. 여기에서는 BOW만 학습할 것.

 텍스트 분석 수행 프로세스 
1. 텍스트 사전 준비작업(텍스트 전처리) : 클렌징, 대/소문자 변경, 특수문자 삭제 등의 클렌징 작업, 단어 등의 토큰화 작업, 의미 없는 단어(stopword) 제거 작업, 어근 추출(Stemming/Lemmatization) 등의 텍스트 정규화 작업 수행 
2. 피처 벡터화/추출: 가공된 텍스트에서 피처를 추출하고 벡터값 할당. BOW나 Word2Vec이 대표적으로 사용되며, BOW는 대표적으로 Count 기반과 TF_IDF 기반 벡터화 있음. 
3. ML 모델 수립 및 학습/예측/평가: 피처 벡터화된 데이터 세트에 ML 모델 적용해 학습/예측 및 평가를 수행

### 텍스트 분석 패키지 
1. NLTK 
2. Gensim: Word2Vec 구현. 토픽 모델링 분야에서 두각을 나타내는 패키지
3. SpaCy

## 02. 텍스트 사전 준비 작업 (텍스트 전처리) - 텍스트 정규화 

### 클렌징 
불필요한 문자, 기호 등을 사전에 제거하는 작업. 

### 텍스트 토큰화 
문서에서 문장을 분리하는 토큰화와 문장에서 단어를 토큰으로 분리하는 단어 토큰화

1. 문장 토큰화: 
마침표나 개행문자(\n) 등의 문장의 마지막을 뜻하는 기호에 따라 분리하는 것이 일반적. 정규표현식에 따른 문장 토큰화도 가능.


In [1]:
# 문장 토큰화
from nltk import sent_tokenize

In [2]:
import nltk 
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all arouns us, here even in this room. \
              You can see it out your window or on your television. \
              You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text=text_sample)

print(type(sentences), len(sentences))
print(sentences)


<class 'list'> 3
['The Matrix is everywhere its all arouns us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


[nltk_data] Downloading package punkt to /Users/harampark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2. 단어 토큰화: 
문장을 단어로 토큰화하는 것. BOW와 같이 단어의 순서가 중요하지 않은 경우 문장 토큰화를 사용하지 않고 단어 토큰화만 사용해도 충분함. 각문장이 가지는 시맨틱적인 의미가 중요한 요소로 사용될 때 사용한다.

In [3]:
from nltk import word_tokenize 

sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


문장 토큰화와 단어 토큰화를 조합해 문서에 대해 모든 단어를 토큰화 해보기

In [4]:
from nltk import word_tokenize, sent_tokenize 

# 여러 개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화하게 만드는 함수 생성
def tokenize_text(text): 
  sentences = sent_tokenize(text)
  word_tokens = [word_tokenize(sentence) for sentence in sentences]
  return word_tokens

word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'arouns', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


이렇게 문장을 단어별로 하나씩 토큰화할 경우 문맥적인 의미는 무시될 수밖에 없음. 이를 조금이라도 해결해보고자 한 것이 **n-gram**이다. 연속된 n개의 단어를 하나의 토큰화 단위로 분리해내는 것. n개 단어 크기 윈도우를 만들어 문장의 처음부터 오른쪽으로 움직이면서 토큰화를 수행함. 

### 스톱 워드 제거 
분석에 큰 의미가 없는 단어를 지칭. 언어별로 이러한 스톱 워드가 목록화돼 있음.

In [5]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harampark/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
print('영어 wtop words 개수:', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 wtop words 개수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [7]:
# stopwords 제거해 의미있는 것만 남겨보자 
import nltk 

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
for sentence in word_tokens: 
  filtered_words=[]
  for word in sentence: 
    word = word.lower()
    if word not in stopwords: 
      filtered_words.append(word)
  all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'arouns', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


### Stemming과 Lemmatization 

문법적 또는 의미적으로 변화하는 단어의 원형을 찾는 것. Lemmatization이 Stemming보다 정교하며 의미론적 기반에서 단어의 원형을 찾는다. Lemmatization은 품사와 같은 문법적인 요소와 더 의미적인 부분을 감안해 정확한 철자로 된 어근 단어를 찾아준다. 그래서 시간이 더 오래 걸림. 

In [9]:
# Stemming 
from nltk.stem import LancasterStemmer 
stemmer = LancasterStemmer()

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
# 최상급은 제대로 인식 못함
print(stemmer.stem('fancier'), stemmer.stem('fanciest'))



work work work
amus amus amus
happy happiest
fant fanciest


In [10]:
# Lemmatization - 정확한 원형 단어 추출위해 단어의 '품사'를 입력
from nltk.stem import WordNetLemmatizer
import nltk 
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'), lemma.lemmatize('amuses','v'), lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'), lemma.lemmatize('happiest','a'))
print(lemma.lemmatize('fancier','a'), lemma.lemmatize('fanciest','a'))


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/harampark/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


amuse amuse amuse
happy happy
fancy fancy


## 03. Bag of Words - BOW 

문서가 가지는 모든 단어(Words)를 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값을 추출하는 모델. 장점은 쉽고 빠른 구축. 그러나 여러가지 제약이 존재. 
1. 문맥 의미(Semantic Context) 반영 부족: 단어의 순서를 고려하지 않아 문장 내에서 문맥적인 의미가 무시됨. 
2. 희소 행렬 문제(희소성, 희소 행렬): BOW로 피처 백터화를 수행하면 희소 행렬 형태의 데이터 세트가 만들어지기 쉬움. 희소행렬(대규모의 칼럼으로 구성된 행렬에서 대부분의 값이 0으로 채워지는 행렬 - 모든 문서에 똑같은 단어가 들어가지 않아 0으로 채워질 확률이 더 높음) ML 알고리즘의 수행시간과 예측 성능을 떨어뜨리기 때문에 희소행렬을 위한 특별한 기법이 마련되어 있음.

### BOW 피처 벡터화 
텍스트를 특정 의미를 가지는 숫자형 값인 벡터값으로 변환해야 하는데, 이러한 변환을 피처 벡터화라고 함. 모든 문서에서 모든 단어를 칼럼 형태로 나열하고 각 문서에서 해당 단어의 횟수나 정규화된 빈도를 값으로 부여하는 데이터 세트 모델로 변경하는 것. 결과적으로 M(문서개수)*N(단어 feature들 n개)의 단어 피처로 이뤄진 행렬을 구성함.

BOW의 피처 벡터화의 2가지 방식
- 카운트 기반의 벡터화: 카운트값이 높을수록 중요한 단어로 인식
- TF-IDF 기반의 벡터화: 개별 문서에서 자주 나타나는 단어에 높은 가중치를 주되, 모든 문서에서 전반적으로 자주 나타나는 단어에 대해서는 페널티를 주는 방식으로 값을 부여. 문서마다 텍스트가 길고 문서의 개수가 많은 경우 카운트 방식보다는 TF-IDF 방식을 사용하는 것이 더 좋은 예측 성능을 보장

### 사이킷런의 Count 및 TF-IDF 벡터화 구현: CountVectorizer, TfidfVectorizer 

#### 사이킷런의 CountVectorizer
카운트 기반의 벡터화를 구현한 클래스. 피처 벡터화만 수행하지 않으며 소문자 일괄 변환, 토큰화, 스톱 워드 필터링 등의 텍스트 전처리도 함께 수행함. fit()과 transform()을 통해 피처 벡터화된 객체를 반환함.

#### 사이킷런의 TfidfVectorizer
TF-IDF 벡터화 클래스 

- max_df: 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외하기위한 파라미터 
- min_df: 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기위한 파라미터 
- max_features: 추출하는 피처의 개수를 제한하며 정수로 값을 지정함.
- stop_words
- n_gram_range: 단어 순서를 보강하기위한 n_gram 범위 설정, 튜플 형태로 (범위 최솟값, 범위 최댓값)을 지정
- analyzer: 피처 추출을 수행한 단위 지정. 디폴트는 'word' 
- token_pattern: 토큰화를 수행하는 정규 표현식 패턴 지정 
- tokenizer: 토큰화를 변도의 커스텀 함수로 이용시 적용 

#### 순서 
1. 사전 데이터 가공: 모든 문자를 소문자로 변환하는 등의 사전 작업 수행
2. 토큰화: 단어 기준 (default), n_gram_range를 반영해 토큰화 수행
3. 텍스트 정규화: stopwords 필터링만 수행. Stemmer, Lemmatize는 CountVectorizer 자체에서는 지원되지 않음. 이를 위한 함수를 만들거나 외부 패키지로 미리 Text Normalization 수행 필요 
4. 피처 벡터화: 다양한 파라미터를 반영해 Token된 단어들을 feature extraction 후 vectorization 적용

### BOW 벡터화를 위한 희소 행렬
사이킷런의 CountVectorizer/TfidfVectorizer를 이용해 텍스트를 피처 단위로 벡터화해 변환하고 CSR 형태의 희소 행렬을 반환함. 이처럼 대규모 행렬의 대부분의 값을 0이 차지하는 행렬을 가리켜 희소행렬이라고 하고, BOW 형태를 가진 언어 모델의 피처 벡터화는 대부분 희소행렬임. 너무 많은 불필요한 0값이 메모리 공간에 할당되기 때문에 연산 시에도 데이터 액세스를 위한 시간이 많이 소요됨. 그래서 적은 메모리 공간을 차지할 수 있도록 변환해야 하는데 대표적인 방법이 **COO 형식과 CSR 형식**이다. 일반적으로 큰 희소 행렬을 저장하고 계산을 수행하는 능력이 CSR 형식이 더 뛰어나기 때문에 CSR을 많이 사용함.

#### 희소 행렬 - COO 형식
COO 형식은 0이 아닌 데이터만 별도의 데이터 배열(array)에 저장하고 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식. 희소행렬 변환을 위해 파이썬 세계에서는 주로 사이파이(scipy) 이용함. 사이파이의 sparse 패키지는 희소행렬 변환을 위한 다양한 모듈을 제공함. 

In [11]:
import numpy as np 

dense = np.array([[3,0,1], [0,2,0]])

In [12]:
from scipy import sparse 

# 0이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 배열로 생성
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용해 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

In [13]:
# 다시 원래의 데이터 행렬로 추출됨 
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])